# CS 439 Final Project: Crime Rate Prediction

**Contributers:** Jenya Pandu (jp2068), Vivian Zhao (vz57), Matt Yun (yy656) \
**Course:** Introduction to Data Science (01:198:439) \
**Professor:** Naina Chaturvedi \
**Semester:** Spring 2025 

### Import necessary libraries:

In [1]:
# import & setup
import pandas as pd
import numpy as np

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# geospatial / mapping
# import folium
# from folium.plugins import HeatMap

### Load and inspect data:

In [2]:
df = pd.read_csv("Chicago-Crime-Data.csv", low_memory=False)
print(df.shape)

(335412, 22)


In [3]:
df.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,5741943,HN549294,08/25/2007 09:22:18 AM,074XX N ROGERS AVE,0560,ASSAULT,SIMPLE,OTHER,False,False,...,49.0,1.0,08A,NaN,NaN,2007.0,08/17/2015 03:03:40 PM,NaN,NaN,NaN
1,25953,JE240540,05/24/2021 03:06:00 PM,020XX N LARAMIE AVE,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,True,False,...,36.0,19.0,01A,1141387.0,1913179.0,2021.0,11/18/2023 03:39:49 PM,41.917838,-87.755969,"(41.917838056, -87.755968972)"
2,26038,JE279849,06/26/2021 09:24:00 AM,062XX N MC CORMICK RD,0110,HOMICIDE,FIRST DEGREE MURDER,PARKING LOT,True,False,...,50.0,13.0,01A,1152781.0,1941458.0,2021.0,11/18/2023 03:39:49 PM,41.995219,-87.713355,"(41.995219444, -87.713354912)"
3,13279676,JG507211,11/09/2023 07:30:00 AM,019XX W BYRON ST,0620,BURGLARY,UNLAWFUL ENTRY,APARTMENT,False,False,...,47.0,5.0,05,1162518.0,1925906.0,2023.0,11/18/2023 03:39:49 PM,41.952345,-87.677975,"(41.952345086, -87.677975059)"
4,13274752,JG501049,11/12/2023 07:59:00 AM,086XX S COTTAGE GROVE AVE,0454,BATTERY,"AGGRAVATED P.O. - HANDS, FISTS, FEET, NO / MIN...",SMALL RETAIL STORE,True,False,...,6.0,44.0,08B,1183071.0,1847869.0,2023.0,12/09/2023 03:41:24 PM,41.737751,-87.604856,"(41.737750767, -87.604855911)"


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335412 entries, 0 to 335411
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   ID                    335412 non-null  int64  
 1   Case Number           335412 non-null  object 
 2   Date                  335412 non-null  object 
 3   Block                 335411 non-null  object 
 4   IUCR                  335411 non-null  object 
 5   Primary Type          335411 non-null  object 
 6   Description           335411 non-null  object 
 7   Location Description  333781 non-null  object 
 8   Arrest                335411 non-null  object 
 9   Domestic              335411 non-null  object 
 10  Beat                  335411 non-null  float64
 11  District              335411 non-null  float64
 12  Ward                  205431 non-null  float64
 13  Community Area        205983 non-null  float64
 14  FBI Code              335411 non-null  object 
 15  

### Data Wrangling: 

In [12]:
def preprocess(data):
    df = data.copy()
    
    df = df.drop_duplicates()
    df = df.dropna()
    df = df.reset_index(drop=True)

    # remove unnecessary cols
    df = df.drop(columns=['ID', 'Case Number', 'Year',
                          'Block', 'IUCR', 'Description', 'Updated On',
                          'X Coordinate', 'Y Coordinate', 'Location'])
    
    return df

clean_df = preprocess(df)
display(clean_df)

,Date,Primary Type,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,Latitude,Longitude
0,05/24/2021 03:06:00 PM,HOMICIDE,STREET,True,False,2515.0,25.0,36.0,19.0,01A,41.917838,-87.755969
1,06/26/2021 09:24:00 AM,HOMICIDE,PARKING LOT,True,False,1711.0,17.0,50.0,13.0,01A,41.995219,-87.713355
2,11/09/2023 07:30:00 AM,BURGLARY,APARTMENT,False,False,1922.0,19.0,47.0,5.0,05,41.952345,-87.677975
3,11/12/2023 07:59:00 AM,BATTERY,SMALL RETAIL STORE,True,False,632.0,6.0,6.0,44.0,08B,41.737751,-87.604856
4,09/06/2023 05:00:00 PM,CRIMINAL DAMAGE,PARKING LOT / GARAGE (NON RESIDENTIAL),False,False,122.0,1.0,42.0,32.0,14,41.886018,-87.633938
...,...,...,...,...,...,...,...,...,...,...,...,...
199417,01/06/2016 05:27:00 AM,HOMICIDE,STREET,False,False,933.0,9.0,3.0,61.0,01A,41.808213,-87.660094
199418,11/04/2005 10:30:00 PM,HOMICIDE,STREET,True,False,1131.0,11.0,24.0,26.0,01A,41.871020,-87.736472
199419,11/30/2012 11:50:00 PM,HOMICIDE,STREET,True,False,2033.0,20.0,48.0,3.0,01A,41.974165,-87.657757
199420,07/22/2001 05:00:00 PM,MOTOR VEHICLE THEFT,PARKING LOT/GARAGE(NON.RESID.),False,False,825.0,8.0,15.0,66.0,07,41.782590,-87.683589


### (Fill in):